In [1]:
import sys
sys.path.append("/path/to/pyWebREPL/")
from pyWebREPL import WEBREPL

In [2]:
webrepl = WEBREPL(host='192.168.12.219', password='123456')


WebREPL connected
>>> 


### Blink the on-board LED

In [3]:
webrepl.send('''
import time  
from machine import Pin  
PIN_BUILTIN_LED = 2 
p2 = Pin(PIN_BUILTIN_LED, Pin.OUT, value=1)   
p2.off()  # Turn on the on-board LED            
time.sleep(0.5) 
p2.on()   # Turn off the on-board LED   
''')
print(webrepl.recv())

6
paste mode; Ctrl-C to cancel, Ctrl-D to finish
=== import time  
=== from machine import Pin  
=== PIN_BUILTIN_LED = 2 
=== p2 = Pin(PIN_BUILTIN_LED, Pin.OUT, value=1)   
=== p2.off()  # Turn on the on-board LED            
=== time.sleep(0.5) 
=== p2.on()   # Turn off the on-board LED
=== 
>>> 


### Test Block 1

In [4]:
webrepl.send('''
if 1:
    print('Hello')
    if 1:
        print('world')
''')
print(webrepl.recv())

6
paste mode; Ctrl-C to cancel, Ctrl-D to finish
=== if 1:
===     print('Hello')
===     if 1:
===         print('world')
=== 
Hello
world
>>> 


### Test Block 2

In [5]:
webrepl.send('''
for i in range(4):
    print(i)
''')
print(webrepl.recv())

6
paste mode; Ctrl-C to cancel, Ctrl-D to finish
=== for i in range(4):
===     print(i)
=== 
0
1
2
3
>>> 


### Test Block 3

In [6]:
code = '''fw = open('test.py', 'w')
fw.write('import time\\n')                
fw.write('count = 0\\n')
fw.write('for i in range(20):\\n')              
fw.write('    print(count)\\n')       
fw.write('    count += 1\\n')     
fw.write('    time.sleep(0.5)\\n') 
fw.close()
''' 
webrepl.send(code)
print(webrepl.recv())

6
paste mode; Ctrl-C to cancel, Ctrl-D to finish
=== fw = open('test.py', 'w')
=== fw.write('import time\n')                
=== fw.write('count = 0\n')
=== fw.write('for i in range(20):\n')              
=== fw.write('    print(count)\n')       
=== fw.write('    count += 1\n')     
=== fw.write('    time.sleep(0.5)\n') 
=== fw.close()
=== 
12
10
20
17
15
20
>>> 


In [7]:
code = '''
exec(open("test.py").read())
''' 
webrepl.send(code)
print(webrepl.recv())

6
paste mode; Ctrl-C to cancel, Ctrl-D to finish
=== exec(open("test.py").read())
=== 
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
>>> 


In [8]:
webrepl.close()